<a href="https://colab.research.google.com/github/Maverick29rus/homework/blob/master/%228_%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D1%8F%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Навигация по уроку**

1. [Библиотеки визуализации данных в Python](https://colab.research.google.com/drive/1IbLhzlqiJhhPAErmdQ9wuIUdDaSRlNUF)
2. [Задача об акциях Tesla](https://colab.research.google.com/drive/1jukOs54u301WtyQS-SbRxWxDstTCjEBq)
3. Домашняя работа

Используя датасет о стоимости акций Сбербанка с 01.01.2013 года:

https://storage.yandexcloud.net/academy.ai/SBER.csv

визуализируйте индикатор "Полосы Боллинджера", проанализируйте график, и предложите вариант торговли акциями Сбербанка с помощью этого инструмента.

**Подсказка.**
Индикатор выглядит как полоса из трех линий:

* линия посередине — это простая скользящая средняя (SMA) с периодом `ma_size`, обычно около 20 дней;

* верхняя и нижняя линии (BB) — построены на основе SMА, но к нему добавлено стреднеквадратичное отклонение:

```
 SMA = data['close'].rolling(ma_size).mean()
 BB_UP = SMA + data['close'].rolling(ma_size).std() * bol_size
 BB_DOWN = SMA - data['close'].rolling(ma_size).std() * bol_size
   
```

где bol_size - ширина коридора, подбирается по графику. Выберите такое его значение, чтобы по графику можно было принимать торговые решения.

In [ ]:
# Загрузим все необходимые библиотеки
import pandas as pd
import plotly as py
import plotly.graph_objects as go

In [ ]:
!wget https://storage.yandexcloud.net/academy.ai/SBER.csv

--2024-12-06 08:41:42--  https://storage.yandexcloud.net/academy.ai/SBER.csv
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180264 (176K) [text/csv]
Saving to: ‘SBER.csv.1’

SBER.csv.1          100%[===================>] 176.04K   344KB/s    in 0.5s    

2024-12-06 08:41:44 (344 KB/s) - ‘SBER.csv.1’ saved [180264/180264]



In [ ]:
df = pd.read_csv("./SBER.csv", sep=';') # wget скачал в рабочую директорию файл с датасетом и мы его уже можем использовать
# Дата должна быть в формате datetime для построения временных рядов
df['DATE'] = pd.to_datetime(df['DATE'], format="%Y%m%d")


In [ ]:
display(df.head(), df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2718 entries, 0 to 2717
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   DATE    2718 non-null   datetime64[ns]
 1   OPEN    2718 non-null   float64       
 2   HIGH    2718 non-null   float64       
 3   LOW     2718 non-null   float64       
 4   CLOSE   2718 non-null   float64       
 5   VOL     2718 non-null   int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 127.5 KB


,DATE,OPEN,HIGH,LOW,CLOSE,VOL
0,2013-01-08,96.50,98.50,96.12,98.37,92329970
1,2013-01-09,98.41,98.65,97.81,98.23,59776760
2,2013-01-10,98.35,98.55,97.96,98.41,47627940
3,2013-01-11,98.80,99.76,98.48,99.56,71837780
4,2013-01-14,99.72,101.27,99.57,100.99,83853170


None

# Краткосрочная стратегия

Период скользящей средней равен 20 дней

In [ ]:
ma_size = 20
bol_size = 2
#SMA - расчет скользящих средних
df['SMA_20'] = df['CLOSE'].rolling(ma_size).mean()

#расчет верхней и нижней линии (BB)
df['BB_UP'] = df['SMA_20'] + df['CLOSE'].rolling(ma_size).std() * bol_size
df['BB_DOWN'] = df['SMA_20'] - df['CLOSE'].rolling(ma_size).std() * bol_size


In [ ]:
def add_signal(df):
    # adds two columns to dataframe with buy and sell signals
    buy_list = []
    sell_list = []

    for i in range(len(df['CLOSE'])):
        #if df['CLOSE'][i] > df['BB_UP'][i]:       # sell signal daily
        if df['HIGH'][i] > df['BB_UP'][i]:        # sell signal weekly
            buy_list.append(np.nan)
            sell_list.append(df['CLOSE'][i])
        #elif df['CLOSE'][i] < df['BB_DOWN'][i]:   # buy signal daily
        elif df['LOW'][i] < df['BB_DOWN'][i]:     # buy signal weekly
            buy_list.append(df['CLOSE'][i])
            sell_list.append(np.nan)
        else:
            buy_list.append(np.nan)
            sell_list.append(np.nan)

    buy_list = pd.Series(buy_list, name='Buy')
    sell_list = pd.Series(sell_list, name='Sell')

    df['Buy'] = buy_list
    df['Sell'] = sell_list

    return df

df = add_signal(df)

In [ ]:
fig = go.Figure()
fig.add_scatter(x=df.DATE, y=df.SMA_20, name='SMA 20')
fig.add_scatter(x=df.DATE, y=df.CLOSE, name='CLOSE', opacity=0.3)
fig.add_scatter(x=df.DATE, y=df.BB_UP, name='BB_UP')
fig.add_scatter(x=df.DATE, y=df.BB_DOWN, name='BB_DOWN')
fig.add_scatter(x=df.DATE, y=df.Buy, name='Buy',
                mode='markers',
                marker=dict(color='Green', symbol='arrow-bar-up')
                )
fig.add_scatter(x=df.DATE, y=df.Sell, name='Sell',
                mode='markers',
                marker=dict(color='Red', symbol='arrow-bar-down')
                )
# Обновляем ось Х, добавляя в нее ползунок диапазона и кнопки выбора диапазона
fig.update_xaxes(rangeslider_visible=True, rangeselector=dict(
    buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.update_yaxes(fixedrange=False)
fig.show()

Вывод: В общем случае по графику можно сделать вывод, что акции необходимо продавать когда цена закрытия пересекает верхнюю линию Боллинджера и покупать когда пересекает нижнюю.

# Долгосрочная стратегия

Период скользящей средней равен 220 дней

In [ ]:
ma_size = 220
bol_size = 2
#SMA - расчет скользящих средних
df['SMA_20'] = df['CLOSE'].rolling(ma_size).mean()

#расчет верхней и нижней линии (BB)
df['BB_UP'] = df['SMA_20'] + df['CLOSE'].rolling(ma_size).std() * bol_size
df['BB_DOWN'] = df['SMA_20'] - df['CLOSE'].rolling(ma_size).std() * bol_size

df = add_signal(df)


In [ ]:
fig = go.Figure()
fig.add_scatter(x=df.DATE, y=df.SMA_20, name='SMA 220')
fig.add_scatter(x=df.DATE, y=df.CLOSE, name='CLOSE', opacity=0.3)
fig.add_scatter(x=df.DATE, y=df.BB_UP, name='BB_UP')
fig.add_scatter(x=df.DATE, y=df.BB_DOWN, name='BB_DOWN')
fig.add_scatter(x=df.DATE, y=df.Buy, name='Buy',
                mode='markers',
                marker=dict(color='Green', symbol='arrow-bar-up')
                )
fig.add_scatter(x=df.DATE, y=df.Sell, name='Sell',
                mode='markers',
                marker=dict(color='Red', symbol='arrow-bar-down')
                )
# Обновляем ось Х, добавляя в нее ползунок диапазона и кнопки выбора диапазона
fig.update_xaxes(rangeslider_visible=True, rangeselector=dict(
    buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.update_yaxes(fixedrange=False)
fig.show()

Вывод: Для долгосрочных стратегий мы видим что общее направление тренда направленно вверх, верхняя и нижняя лянии разошлись и только начинают сужаться, что свидетельствует о продолжении тренда и возможном в скором времени изменении направления. Для тех кто хочет продавать имеет смысл еще подождать пока тренд не дойдет до пика.